## Notebook Demonstrating the Use of Multiple Tools with LangChain

This example showcases how an LLM can generate scripts based on user input, save them to the appropriate files, and execute the resulting commands.

**Note:** *Use with caution.* The notebook executes commands generated by the LLM, and there is currently no safety or validation layer. For this reason, it is strongly recommended to run the notebook inside a Docker environment for proper sandboxing.

In [1]:
import os
import subprocess
import argparse
from datetime import datetime
from typing import Annotated

from dotenv import load_dotenv
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# Load OPENAI_API_KEY from .env file
load_dotenv()

True

### Defining a Tool with the LangChain `@tool` Decorator

Below is an example of a tool that generates a Python script from a natural-language request and saves it to disk.

This tool accepts two arguments:

* **`user_request`** – A natural-language description of the script you want to generate. This usually describes the task the generated Python script should perform.
* **`output_dir`** – The directory where the script will be saved. By default, scripts are written to `generated_scripts`.

Inside the function, we implement the logic required to create the directory (if needed), generate the script using an LLM, clean the output, construct a safe filename, and write the final Python file to disk.

**Note:**
When returning from a tool, we typically do not return only the raw value. Instead, we include a message describing what the tool accomplished and what type of output it produced. This helps the LLM correctly interpret the tool’s result and decide on its next action.

**References:**

* LangChain Documentation – Tools: [https://docs.langchain.com/oss/python/langchain/tools](https://docs.langchain.com/oss/python/langchain/tools)

In [2]:
@tool
def generate_and_save_script(
    user_request: Annotated[
        str,
        "Natural language description of the script you want to generate. "
        "Be as specific as possible (e.g., 'read a CSV file and print the first 5 rows')."
    ],
    output_dir: Annotated[
        str,
        "Directory where the script will be saved (default: generated_scripts)"
    ] = "generated_scripts"
) -> str:
    """
    Generates a complete Python script from a natural language request and saves it to a file.
    Returns the absolute path of the saved script.
    """
    # 1. Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # 2. Initialize LLM
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    # 3. Build prompt
    prompt = f"""You are an expert Python developer. Write a **complete, runnable Python script** that fulfills the following request:

{user_request}

Requirements:
- Include `if __name__ == "__main__":` guard.
- Add helpful docstrings and comments.
- Use only standard library unless the request explicitly needs a third-party package.
- Do **not** include any explanation outside the code.
- Return the code **exactly** as it should be written to a .py file.
"""
    
    # 4. Call LLM
    response = llm.invoke([HumanMessage(content=prompt)])
    script_code = response.content.strip()
    
    # 5. Clean markdown (if any)
    if script_code.startswith("```python"):
        script_code = script_code[len("```python"):].lstrip()
    if script_code.endswith("```"):
        script_code = script_code[:-3].rstrip()
    script_code = script_code.strip()

    # 6. Generate safe filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    safe_name = "".join(c if c.isalnum() else "_" for c in user_request.lower())[:50]
    filename = f"script_{timestamp}_{safe_name}.py"
    
    filepath = os.path.abspath(os.path.join(output_dir, filename))

    # 8. Write file
    with open(filepath, "w", encoding="utf-8") as fp:
        fp.write(script_code)

    # 9. Return result
    return f"Script saved to: {filepath}"

### Defining a Tool to Execute Shell Commands

Next, we create a second tool whose purpose is to execute shell commands. While the previous tool generates and saves Python scripts, this tool is responsible for running those scripts—or any other valid shell command.

It accepts a single argument:

* **`cmd`** – A valid bash command to execute. This can include absolute paths, pipes, redirects, or any other shell features.

The tool runs the command in a real shell environment, captures both **stdout** and **stderr**, handles timeouts and common errors, and returns a detailed result message describing the command’s output or failure.

**References:**

* Python `subprocess` module: [https://docs.python.org/3/library/subprocess.html](https://docs.python.org/3/library/subprocess.html)

In [3]:
@tool
def run_script(
    cmd: Annotated[
        str,
        "Bash command line to execute. "
        "Must be a valid shell command. "
        "Use absolute paths when possible. "
        "Example: 'python my_script.py'"
    ]
) -> str:
    """
    Executes a shell command and returns the output.

    - Captures both **stdout** and **stderr**.
    - Returns **full output** on success.
    - Returns **error message** on failure.
    - Runs in a **real shell** (so pipes, redirects, etc. work).

    Args:
        cmd: The exact command to run in the shell.

    Returns:
        str: Command output or error message.
    """
    try:

        print(f"[INFO] CMD: {cmd}")
        # Use shell=True to support pipes, &&, >, etc.
        # capture_output=True gets both stdout and stderr
        result = subprocess.run(
            cmd,
            shell=True,
            capture_output=True,
            text=True,
            encoding="utf-8",
            timeout=60,  # Prevent hanging
            cwd=os.getcwd()  # Run in current working directory
        )

        if result.returncode == 0:
            output = result.stdout.strip()
            return f"Success:\n{output}" if output else "Command executed successfully (no output)."
        else:
            error = result.stderr.strip()
            return f"Failed (exit code {result.returncode}):\n{error}"

    except subprocess.TimeoutExpired:
        return "Error: Command timed out after 60 seconds."
    except FileNotFoundError:
        return f"Error: Command not found: {cmd.split()[0]}"
    except Exception as e:
        return f"Unexpected error: {str(e)}"

### That the LLM sees when we register the tool with it

In [4]:
generate_and_save_script.args_schema.model_json_schema()

{'description': 'Generates a complete Python script from a natural language request and saves it to a file.\nReturns the absolute path of the saved script.',
 'properties': {'user_request': {'description': "Natural language description of the script you want to generate. Be as specific as possible (e.g., 'read a CSV file and print the first 5 rows').",
   'title': 'User Request',
   'type': 'string'},
  'output_dir': {'default': 'generated_scripts',
   'description': 'Directory where the script will be saved (default: generated_scripts)',
   'title': 'Output Dir',
   'type': 'string'}},
 'required': ['user_request'],
 'title': 'generate_and_save_script',
 'type': 'object'}

### Creating the Agent and Registering Tools

Next, we create an agent and register our two tools:

1. **`generate_and_save_script`** – to create and store a Python script, and
2. **`run_script`** – to execute shell commands.

We then provide a prompt instructing the agent to generate a script that lists all files in the current directory, saves the list to `output.txt`, executes the script, and finally displays the file’s contents.

The agent analyzes the task and decides to first call the `generate_and_save_script` tool. After the script is created, the tool returns the absolute path to the saved file. Using this information, the agent determines the appropriate shell command and invokes the `run_script` tool to execute it.

Once the command finishes running, the agent recognizes that the task is complete and returns the final result to the user.

**Note:**
This example does **not** include any safety checks on the generated shell commands, which can be risky. In a production environment, command validation and sandboxing are essential. For demonstration purposes, these checks are omitted here.


In [5]:
from langgraph.prebuilt import create_react_agent
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

human_prompt = "Create a script to list all the files in the current directory and save the list to output.txt; then run the script in bash to generate output.txt and display its contents."
# human_prompt = 
agent = create_agent(llm, tools=[generate_and_save_script, run_script])
response = agent.invoke({
    "messages": [("human", human_prompt)]
})
print(response["messages"][-1].content)

[INFO] CMD: bash /app/generated_scripts/script_20251114_103622_create_a_script_to_list_all_the_files_in_the_curre.py
[INFO] CMD: cat output.txt
[INFO] CMD: bash /app/generated_scripts/script_20251114_103636_create_a_bash_script_to_list_all_the_files_in_the_.py
[INFO] CMD: cat output.txt
[INFO] CMD: cat output.txt
[INFO] CMD: bash /app/generated_scripts/script_20251114_103648_create_a_bash_script_that_lists_all_the_files_in_t.py
[INFO] CMD: bash /app/generated_scripts/script_20251114_103659____bin_bash_ls___output_txt_.py
[INFO] CMD: cat output.txt
[INFO] CMD: bash /app/generated_scripts/script_20251114_103712_list_all_files_in_the_current_directory_and_save_t.py
[INFO] CMD: ls > output.txt
[INFO] CMD: cat output.txt
The command was successfully executed, and the list of files in the current directory has been saved to `output.txt`. Here are the contents of `output.txt`:

```
Dockerfile
README.md
generate_and_run_with_tools.ipynb
generated_scripts
output.txt
requirements.txt
```

If you

### Understanding the Agent's Process and Outputs

In this example, you can track the agent's steps through the **logs** and understand how it attempts to complete the task. Here's a breakdown:

1. The LLM initially generates a script as requested, but it treats the generated Python script as an executable. As a result, it encounters errors trying to run the script directly. This leads to failed attempts where the agent keeps trying to execute the non-executable Python file.

2. The process is iterative. When the agent realizes it is facing errors with the generated Python script, it continues by calling the `generate_and_save_script` tool again to create a new version of the script, then attempts to run it once more using the `run_script` tool.

3. The agent seems to repeatedly generate new scripts because it does not have an understanding of the code previously generated. This is because the agent does not have memory of the code generation step—it only reacts based on the most recent request to the tool. Each time it runs, it effectively "forgets" the previous scripts and re-generates a new one.

4. After several iterations, the agent "cheats" by switching to a much simpler solution: it directly runs the `ls > output.txt` command, which lists all files in the current directory and saves the result to `output.txt`. It then uses `cat output.txt` to show the contents of the file, successfully completing the task.

5. **Key insight:** The agent’s failure to correct the script is because it doesn’t have knowledge of the code it’s generating (or previous executions), which limits its ability to adapt and fix errors. The tools it calls (like `generate_and_save_script` and `run_script`) work in isolation and do not share state between invocations.

In short, the agent can perform tasks iteratively, but without a feedback loop to learn from the generated code, it ends up making repeated attempts until it finally resorts to a simpler command to achieve the desired result.

Happy Hacking 💻🚀😄

In [6]:
for msg in response["messages"]:
    print(msg)
    print("-----------------")

content='Create a script to list all the files in the current directory and save the list to output.txt; then run the script in bash to generate output.txt and display its contents.' additional_kwargs={} response_metadata={} id='c2eb61fa-2616-45ba-9588-f82c05a0bcbc'
-----------------
content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 283, 'total_tokens': 318, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CblS1L4lCOPYVR2huV6p0AaTzVGO2', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--7b304261-1fa6-4564-97ec-f3b7e32534d2-0' tool_calls=[{'name': 'generate_and_save_script', 'args': {'user_request': 'C